In [27]:
import requests
import json
import os
import traceback
import datetime
import pandas as pd
import numpy as np

# Constants
UNISWAP_V3_SUBGRAPH_URL = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

# GraphQL query template
query_template = """
query fetch_poolDayDatas($poolAddress: String!, $cursorID: ID) {
  poolDayDatas(
    where: {pool: $poolAddress, id_gt: $cursorID}
    first: 1000
    orderBy: id
    orderDirection: asc
  ) {
    id
    high
    low
    tvlUSD
    txCount
    volumeUSD
    date
    open
    tick    
  }
}
"""

# Function to execute a GraphQL query
def execute_graphql_query(subgraph_url, query, variables):
    response = requests.post(subgraph_url, json={'query': query, 'variables': variables})
    response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
    json_response = response.json()
    if 'errors' in json_response:
        raise Exception("GraphQL Error: " + json.dumps(json_response['errors']))
    return json_response['data']['poolDayDatas']

# Function to save data to a JSON file
def save_to_file(data, file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile, indent=2)

# Function to paginate through and fetch all pool day data
def fetch_all_pool_day_data(subgraph_url, query, pool_address):
    all_data = []
    last_id = ""
    
    while True:
        print(f"Fetching pool day data after ID {last_id}")
        batch = execute_graphql_query(subgraph_url, query, {'poolAddress': pool_address.lower(), 'cursorID': last_id})
        if not batch:
            break  # No more data to fetch
        all_data.extend(batch)
        last_id = batch[-1]['id']
    
    return all_data

def timestamp_to_date(json_data):
    for entry in json_data:
        # Assuming 'date' is the key for the timestamp in the JSON object
        timestamp = int(entry['date'])
        # Convert Unix timestamp to datetime object
        date_time = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string in the format 'YYYY-MM-DD'
        entry['date'] = date_time.strftime('%Y-%m-%d')
    return json_data

def insert_missing_days(data):
    if not data:
        return data  # If the data list is empty, return it as is.

    # Sort data by date just in case it's not already sorted
    data.sort(key=lambda x: x['date'])

    # Convert the first and last date in the data to datetime objects
    first_date = datetime.datetime.strptime(data[0]['date'], '%Y-%m-%d')
    last_date = datetime.datetime.strptime(data[-1]['date'], '%Y-%m-%d')

    # Calculate the expected number of days based on the date range
    expected_days = (last_date - first_date).days + 1
    current_date = first_date

    # Placeholder for the new dataset with no missing days
    new_data = []

    for i in range(expected_days):
        # Convert current_date back to string to compare with data
        current_date_str = current_date.strftime('%Y-%m-%d')
        
        # If the date matches, append the data and increment the index
        if data and data[0]['date'] == current_date_str:
            new_data.append(data.pop(0))
        else:
            print(f"Inserting data at {current_date_str}")
            # Insert a new placeholder object for the missing date
            new_data.append({
                'id': None,
                'high': None,
                'low': None,
                'tvlUSD': None,
                'txCount': None,
                'volumeUSD': None,
                'date': current_date_str,
                'open': None,
                'tick': None,
            })

        # Increment the current_date by one day
        current_date += datetime.timedelta(days=1)

    return new_data

def interpolate_json_data(json_data, keys_not_to_interpolate):
    # Convert JSON to DataFrame
    df = pd.DataFrame(json_data)
    
    # Save the columns that should not be interpolated
    non_interpolated_data = df[keys_not_to_interpolate].copy()

    # Replace 'null' string and 0 values with np.nan for the rest
    for key in df.columns.difference(keys_not_to_interpolate):
        df[key] = df[key].replace('null', np.nan).replace('0', np.nan).astype(float)

    # Perform the interpolation on allowed keys
    df_interpolated = df.interpolate(method='linear', limit_direction='forward', axis=0)

    # Fill first row NaNs if there are any after interpolation
    df_interpolated.fillna(method='bfill', inplace=True)
    
    # Put back the non-interpolated data
    for key in keys_not_to_interpolate:
        df_interpolated[key] = non_interpolated_data[key]

    # Convert DataFrame back to JSON
    interpolated_json_data = df_interpolated.to_dict(orient='records')
    return interpolated_json_data

def reverse_prices(data):
    for entry in data:
        for field in ['high', 'low', 'open', 'close']:
            if entry[field] is not None and entry[field] != 0:
                entry[field] = 1 / entry[field]
    return data

def process_pool_day_data(subgraph_url, query, pool_address):
    data = fetch_all_pool_day_data(subgraph_url, query, pool_address)

    # Add the "close" attribute to each day
    for i in range(len(data) - 1):
        data[i]['close'] = data[i + 1]['open']
    data[-1]['close'] = 0  # Handle the last data point

    data = timestamp_to_date(data)
    data = insert_missing_days(data)

    keys_not_to_interpolate = ['id', 'date']
    data = interpolate_json_data(data, keys_not_to_interpolate)

    return data

def fetch_and_process_all_pools(pool_list, subgraph_url, query, output_folder):
    for pool in pool_list:
        pool_address = pool['pool_address']
        reversed_flag = pool['reversed']
        output_file_path = os.path.join(output_folder, f"poolDayData_{pool_address}.json")

        try:
            data = process_pool_day_data(subgraph_url, query, pool_address)
            if reversed_flag:
                data = reverse_prices(data)
            save_to_file(data, output_file_path)
            print(f"Data for pool {pool['pair']} processed and saved to {output_file_path}")
        except Exception as e:
            print(f"An error occurred while processing pool {pool['pair']}:")
            traceback.print_exc()

def main():
    pool_list = [
        {
            "pair": "USDC-ETH",
            "pool_address": "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640",
            "reversed": False
        },
        {
            "pair": "WBTC-USDC",
            "pool_address": "0x99ac8cA7087fA4A2A1FB6357269965A2014ABc35",
            "reversed": True
        },
        {
            "pair": "UNI-USDC",
            "pool_address": "0xD0fC8bA7E267f2bc56044A7715A489d851dC6D78",
            "reversed": True
        },
        {
            "pair": "LDO-USDC",
            "pool_address": "0x78235D08B2aE7a3E00184329212a4d7AcD2F9985",
            "reversed": True
        },
        {
            "pair": "LINK-USDC",
            "pool_address": "0xFAD57d2039C21811C8F2B5D5B65308aa99D31559",
            "reversed": True
        },
        {
            "pair": "MATIC-USDC",
            "pool_address": "0x07A6E955bA4345BAe83Ac2A6fAa771fddd8A2011",
            "reversed": True
        },
        {
            "pair": "USDC-GRT",
            "pool_address": "0xB06E7Ed37CFA8F0f2888355DD1913e45412798c5",
            "reversed": False
        },
    ]
    output_folder = "data"  # Replace with your actual path
    fetch_and_process_all_pools(pool_list, UNISWAP_V3_SUBGRAPH_URL, query_template, output_folder)

# Call the main function
if __name__ == "__main__":
    main()

Fetching pool day data after ID 
Fetching pool day data after ID 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-19669
Data for pool USDC-ETH processed and saved to data/poolDayData_0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640.json
Fetching pool day data after ID 
Fetching pool day data after ID 0x99ac8ca7087fa4a2a1fb6357269965a2014abc35-19669
Data for pool WBTC-USDC processed and saved to data/poolDayData_0x99ac8cA7087fA4A2A1FB6357269965A2014ABc35.json
Fetching pool day data after ID 
Fetching pool day data after ID 0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78-19669
Data for pool UNI-USDC processed and saved to data/poolDayData_0xD0fC8bA7E267f2bc56044A7715A489d851dC6D78.json
Fetching pool day data after ID 
Fetching pool day data after ID 0x78235d08b2ae7a3e00184329212a4d7acd2f9985-19669
Inserting data at 2022-06-14
Inserting data at 2022-06-15
Inserting data at 2022-06-16
Inserting data at 2022-06-18
Inserting data at 2022-06-19
Inserting data at 2022-06-20
Inserting data at 2022-06-21
Inserti